# Frequent Subtree Counting in Random Forests

The goal of this project is to compress the generated source code of decision tree classifiers on embedded devices.
Therefore, as a first step, we investigate for several trained random forests, whether they have certain frequent subtrees in common.
Such subtrees might be implemented by a function which is called several times in the corresponding places of the decision trees. 
This can decrease the code size of the generated embedded-c source files and executables.

## Datasets
There are several datasets.
At the moment, however, I'll experiment only with 'adult' and 'wine-quality'.
The data are given as json files of the following names:

In [5]:
ls forests/*/text/*.json

forests/adult/text/DT_10.json  forests/wine-quality/text/DT_10.json
forests/adult/text/DT_15.json  forests/wine-quality/text/DT_15.json
forests/adult/text/DT_1.json   forests/wine-quality/text/DT_1.json
forests/adult/text/DT_20.json  forests/wine-quality/text/DT_20.json
forests/adult/text/DT_5.json   forests/wine-quality/text/DT_5.json
forests/adult/text/ET_10.json  forests/wine-quality/text/ET_10.json
forests/adult/text/ET_15.json  forests/wine-quality/text/ET_15.json
forests/adult/text/ET_1.json   forests/wine-quality/text/ET_1.json
forests/adult/text/ET_20.json  forests/wine-quality/text/ET_20.json
forests/adult/text/ET_5.json   forests/wine-quality/text/ET_5.json
forests/adult/text/RF_10.json  forests/wine-quality/text/RF_10.json
forests/adult/text/RF_15.json  forests/wine-quality/text/RF_15.json
forests/adult/text/RF_1.json   forests/wine-quality/text/RF_1.json
forests/adult/text/RF_20.json  forests/wine-quality/text/RF_20.json
forests/adult/text/RF_5.json   forests/wine-quality/t

The filenames XX_n.json mean:
- XX:
  - DT decision tree
  - RF random forest with 25 trees
  - ET 'extremely random trees'
- n: depth of the trees

Each tree given here (whether as a decision tree or as a component in a random forest) is a rooted, binary, ordered tree.

## Convert the Data

We have written a few scripts to convert from json to the format required by the frequent subgraph mining software.

In [7]:
ls ./*.py

./json2graphNoLeafEdges.py  ./json2graphWithLeafEdges.py


See the documentation of the scripts to check what they are doing:

In [12]:
for f in ./*.py; do
    echo ${f}
    grep '^#' < ${f}
    echo
done

./json2graphNoLeafEdges.py
## This script creates a graph database from the decision trees in Sebastians json Format as follows:
# - the root vertex of the tree has index 1 (counting starts with 1)
# - each vertex is labeled by its split feature or by 'leaf'
# - each edge is labeled either 'leftChild' or 'rightChild'
# - there are no edges containing 'leaf' vertices
#
# It follows that the connected components resulting from a single decision tree are several isolated vertices labeled 'leaf' 
# and a tree containing all the split vertices.

./json2graphWithLeafEdges.py
## This script creates a graph database from the decision trees in Sebastians json Format as follows:
# - the root vertex of the tree has index 1 (counting starts with 1)
# - each vertex is labeled by its split feature or by 'leaf'
# - each edge is labeled either 'leftChild' or 'rightChild'
# - for each edge in a decision tree there is an edge in the graph database
#
# It follows that the resulting trees in the database 

In [13]:
mkdir forests/adult/WithLeafEdges/
mkdir forests/adult/NoLeafEdges/

mkdir forests/wine-quality/WithLeafEdges/
mkdir forests/wine-quality/NoLeafEdges/

In [28]:
for dataset in adult wine-quality; do
    for f in forests/${dataset}/text/*.json; do
        echo ${f} '->' `basename ${f} .json`.graph
        python json2graphWithLeafEdges.py ${f} > forests/${dataset}/WithLeafEdges/`basename ${f} .json`.graph
        python json2graphNoLeafEdges.py ${f} > forests/${dataset}/NoLeafEdges/`basename ${f} .json`.graph
    done
done

forests/adult/text/DT_10.json -> DT_10.graph
forests/adult/text/DT_15.json -> DT_15.graph
forests/adult/text/DT_1.json -> DT_1.graph
forests/adult/text/DT_20.json -> DT_20.graph
forests/adult/text/DT_5.json -> DT_5.graph
forests/adult/text/ET_10.json -> ET_10.graph
forests/adult/text/ET_15.json -> ET_15.graph
forests/adult/text/ET_1.json -> ET_1.graph
forests/adult/text/ET_20.json -> ET_20.graph
forests/adult/text/ET_5.json -> ET_5.graph
forests/adult/text/RF_10.json -> RF_10.graph
forests/adult/text/RF_15.json -> RF_15.graph
forests/adult/text/RF_1.json -> RF_1.graph
forests/adult/text/RF_20.json -> RF_20.graph
forests/adult/text/RF_5.json -> RF_5.graph
forests/wine-quality/text/DT_10.json -> DT_10.graph
forests/wine-quality/text/DT_15.json -> DT_15.graph
forests/wine-quality/text/DT_1.json -> DT_1.graph
forests/wine-quality/text/DT_20.json -> DT_20.graph
forests/wine-quality/text/DT_5.json -> DT_5.graph
forests/wine-quality/text/ET_10.json -> ET_10.graph
forests/wine-quality/text/ET_

## Find Frequent Subtrees

To be able to meaningfully find frequent subtrees here, we actually need to do two things in the graph mining executable:
- make the algorithm able to deal with rooted trees in a meaningful way
- make the algorithm output at least the root vertex of an embedding (if it exists for a given transaction tree) instead of just 'there is a mapping'

### Find Frequent Undirected Trees

Regardless of the above, I'll first check, how many undirected frequent trees we can find in the 'unrooted variant' of the random forests.
That is: We consider the undirected graphs arising from the rooted decision trees by 'forgetting' the root. 
If there exists a subgraph isomorphism in the rooted variant, then this implies that there exists a subgraph isomorphism in this undirected version. 
Hence, the number of frequent undirected subtrees is an upper bound on the number of the frequent directed subtrees.

In [3]:
# create output directories
mkdir forests/undirectedFrequentTrees/
for dataset in adult wine-quality; do
    mkdir forests/undirectedFrequentTrees/${dataset}/
    for variant in WithLeafEdges NoLeafEdges; do
        mkdir forests/undirectedFrequentTrees/${dataset}/${variant}/
    done
done

In [1]:
./lwg -h

This is a frequent subtree mining tool.
Implemented by Pascal Welke starting in 2016.

This program computes and outputs frequent subtrees according to several
definitions of that notion and feature representations of the mined graphs.

usage: ./lwg [options] [FILE]

If no FILE argument is given or FILE is - the program reads from stdin.
It always prints to stdout (embedding info) and stderr (statistics).


Options:
-h:           print this possibly helpful information.

-t THRESHOLD: Minimum absolute support threshold in the graph database

-p SIZE:      Maximum size (number of vertices) of patterns returned

-o FILE:      output the frequent subtrees in this file

-i VALUE:     Some embedding operators require a parameter that might be
              a float between 0.0 and 1.0 or an integer >=1, depending 
              on the operator.
              
-r VALUE:     Initialize the random number generator with seed VALUE. If
              not specified, random generator is seeded accor

             evaluation of the embedding operator, a (possibly) novel 
             sample of local spanning trees is drawn. This destroys the 
             apriori-property of the embedding operator.
             
           localEasySamplingSlow:
         
           localEasySamplingIndependent:

 


In [5]:
for dataset in adult wine-quality; do
    for variant in WithLeafEdges NoLeafEdges; do
        for f in forests/${dataset}/${variant}/*.graph; do
            for threshold in `seq 25 -1 2`; do
            
                echo "processing threshold ${threshold} for ${f}"
                ./lwg -e subtree -m bfs -t ${threshold} -p 10 \
                  -o forests/undirectedFrequentTrees/${dataset}/${variant}/`basename ${f} .graph`_t${threshold}.patterns \
                  < ${f} \
                  > forests/undirectedFrequentTrees/${dataset}/${variant}/`basename ${f} .graph`_t${threshold}.features \
                  2> forests/undirectedFrequentTrees/${dataset}/${variant}/`basename ${f} .graph`_t${threshold}.logs
                  
            done
        done
    done
done

processing threshold 25 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 24 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 23 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 22 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 21 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 20 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 19 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 18 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 17 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 16 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 15 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 14 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 13 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 12 for forests/adult/WithLeafEdges/DT_10.graph
processing threshold 11 for forests/adult/WithLe

processing threshold 23 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 22 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 21 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 20 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 19 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 18 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 17 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 16 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 15 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 14 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 13 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 12 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 11 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 10 for forests/adult/WithLeafEdges/ET_10.graph
processing threshold 9 for forests/adult/WithLea

processing threshold 21 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 20 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 19 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 18 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 17 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 16 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 15 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 14 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 13 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 12 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 11 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 10 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 9 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 8 for forests/adult/WithLeafEdges/RF_10.graph
processing threshold 7 for forests/adult/WithLeafE

processing threshold 19 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 18 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 17 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 16 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 15 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 14 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 13 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 12 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 11 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 10 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 9 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 8 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 7 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 6 for forests/adult/NoLeafEdges/DT_10.graph
processing threshold 5 for forests/adult/NoLeafEdges/DT_10.graph
processing thre

processing threshold 13 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 12 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 11 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 10 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 9 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 8 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 7 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 6 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 5 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 4 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 3 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 2 for forests/adult/NoLeafEdges/ET_10.graph
processing threshold 25 for forests/adult/NoLeafEdges/ET_15.graph
processing threshold 24 for forests/adult/NoLeafEdges/ET_15.graph
processing threshold 23 for forests/adult/NoLeafEdges/ET_15.graph
processing thresho

...

processing threshold 22 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 21 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 20 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 19 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 18 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 17 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 16 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 15 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 14 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 13 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 12 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 11 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 10 for forests/wine-quality/WithLeafEdges/RF_10.graph
processing threshold 9 fo

processing threshold 7 for forests/wine-quality/WithLeafEdges/RF_5.graph
processing threshold 6 for forests/wine-quality/WithLeafEdges/RF_5.graph
processing threshold 5 for forests/wine-quality/WithLeafEdges/RF_5.graph
processing threshold 4 for forests/wine-quality/WithLeafEdges/RF_5.graph
processing threshold 3 for forests/wine-quality/WithLeafEdges/RF_5.graph
processing threshold 2 for forests/wine-quality/WithLeafEdges/RF_5.graph
processing threshold 25 for forests/wine-quality/NoLeafEdges/DT_10.graph
processing threshold 24 for forests/wine-quality/NoLeafEdges/DT_10.graph
processing threshold 23 for forests/wine-quality/NoLeafEdges/DT_10.graph
processing threshold 22 for forests/wine-quality/NoLeafEdges/DT_10.graph
processing threshold 21 for forests/wine-quality/NoLeafEdges/DT_10.graph
processing threshold 20 for forests/wine-quality/NoLeafEdges/DT_10.graph
processing threshold 19 for forests/wine-quality/NoLeafEdges/DT_10.graph
processing threshold 18 for forests/wine-quality/No

processing threshold 13 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 12 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 11 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 10 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 9 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 8 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 7 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 6 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 5 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 4 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 3 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 2 for forests/wine-quality/NoLeafEdges/DT_5.graph
processing threshold 25 for forests/wine-quality/NoLeafEdges/ET_10.graph
processing threshold 24 for forests/wine-quality/NoLeafEdges/ET_10.grap

processing threshold 19 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 18 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 17 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 16 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 15 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 14 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 13 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 12 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 11 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 10 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 9 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 8 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 7 for forests/wine-quality/NoLeafEdges/ET_5.graph
processing threshold 6 for forests/wine-quality/NoLeafEdges/ET_5.gr

processing threshold 25 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 24 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 23 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 22 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 21 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 20 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 19 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 18 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 17 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 16 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 15 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 14 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 13 for forests/wine-quality/NoLeafEdges/RF_5.graph
processing threshold 12 for forests/wine-quality/NoLeafEdges/RF_

### Next Steps

The results of this mining process are plotted in the python3 notebook 'Plotting Results for Undirected Graphs.ipynb'.
Note that the mining process resulting in output 'forests/undirectedFrequentTrees/adult/WithLeafEdges/ER_20_t2.*' did not finish properly and probably got killed due to excessive memory usage while processing patterns of size 6.